For designing data pipeline to ingest the data into graph database with using MovieLens dataset. I will use neo4j as a graph database. I will create 7 tables, 3 of them nodes and others for relationships. You can find details of tables below;

Nodes

    1- ml_users (userId): This data includes users' id and it has only one column. It is created with using "ratings.csv" data. We will create users nodes and it will have relations with movies and genres nodes
    2- ml_movies (movieId, title, rating_mean): This data invludes movies' id and title fields. It is created with using "movies.csv" data. Movies node will have relations with users and genres nodes and it has relationship to itself as well
    3- ml_genres (genres): It is small data it has 19 rows it keeps genres types

Relationships

    1- ml_users_movies (userId, movieId, rating): This data will use create a relationship between users and movies nodes. It is created with using "ratings.csv" data
    2- ml_movies_genres (movieId, genres): This data will use create a relationship between movies and genres nodes. It is created with using "movies.csv" data
    3- ml_users_genres (userId, genres): This data will use create a relationship between users and genres nodes. genres is a calculated field. It includes the most popular genre of the user. To calculate the most popular genre, I use count of the genres. I thought to use ratings but after some checks I decided to use counts instead of ratings
    4- ml_movies_similarity (movieId, sim_movieId, relevance): This is the most critical data in this pipeline. It includes 5 rows for each movies. I calculate similarity through movies. I use 3 groups of similarity and mix them. It will explain in details. 3 groups are tag similarity, genre similarity and rating,year,rating count similarity. To calculate similarity, I use cosine similarity because I think Cosine similarity often allows better semantics since the high dimensionality.

# getting data

In [1]:
import pandas as pd
import numpy as np
import datetime
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
genome_scores_data = pd.read_csv('genome-scores.csv')
movies_data = pd.read_csv('movies.csv')
ratings_data = pd.read_csv('ratings.csv')

In [3]:
genome_scores_data.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [4]:
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings_data.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


# ml_users

In [3]:
users_df = pd.DataFrame(ratings_data['userId'].unique(), columns=['userId'])

In [4]:
users_df.head()

,userId
0,1
1,2
2,3
3,4
4,5


In [11]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138493 entries, 0 to 138492
Data columns (total 1 columns):
userId    138493 non-null int64
dtypes: int64(1)
memory usage: 1.1 MB


In [12]:
#save users data
users_df.to_csv('graphdb/ml_users.csv', sep='|', header=True, index=False)

# ml_movies

In [6]:
movies_df = movies_data.drop('genres', axis = 1)

In [7]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [8]:
#calculate mean of ratings for each movies
agg_rating_avg = ratings_data.groupby(['movieId']).agg({'rating': np.mean}).reset_index()
agg_rating_avg.columns = ['movieId', 'rating_mean']

In [9]:
movies_df = movies_df.merge(agg_rating_avg, left_on='movieId', right_on='movieId', how='left')

In [10]:
movies_df.head()

,movieId,title,rating_mean
0,1,Toy Story (1995),3.921240
1,2,Jumanji (1995),3.211977
2,3,Grumpier Old Men (1995),3.151040
3,4,Waiting to Exhale (1995),2.861393
4,5,Father of the Bride Part II (1995),3.064592


In [11]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27278 entries, 0 to 27277
Data columns (total 3 columns):
movieId        27278 non-null int64
title          27278 non-null object
rating_mean    26744 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 852.4+ KB


In [12]:
#save movies data
movies_df.to_csv('graphdb/ml_movies.csv', sep='|', header=True, index=False)

# ml_genres

In [17]:
genres = [
    "Action",
    "Adventure",
    "Animation",
    "Children",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
    "(no genres listed)"]

In [18]:
genres_df = pd.DataFrame(genres, columns=['genres'])

In [19]:
genres_df.head()

,genres
0,Action
1,Adventure
2,Animation
3,Children
4,Comedy


In [20]:
#save genres data
genres_df.to_csv('graphdb/ml_genres.csv', sep='|', header=True, index=False)

# ml_users_movies

In [21]:
users_movies_df = ratings_data.drop('timestamp', axis = 1)

In [22]:
users_movies_df.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [23]:
users_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 3 columns):
userId     int64
movieId    int64
rating     float64
dtypes: float64(1), int64(2)
memory usage: 457.8 MB


In [24]:
#save users movies data
users_movies_df.to_csv('graphdb/ml_users_movies.csv', sep='|', header=True, index=False)

# ml_movies_genres

In [25]:
movies_genres_df = movies_data.drop('title', axis = 1)

In [26]:
movies_genres_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 2 columns):
movieId    27278 non-null int64
genres     27278 non-null object
dtypes: int64(1), object(1)
memory usage: 426.3+ KB


In [27]:
#define a function to split genres field
def get_movie_genres(movieId):
    movie = movies_genres_df[movies_genres_df['movieId']==movieId]
    genres = movie['genres'].tolist()
    df = pd.DataFrame([b for a in [i.split('|') for i in genres] for b in a], columns=['genres'])
    df.insert(loc=0, column='movieId', value=movieId)
    return df

In [29]:
#create empty df
movies_genres=pd.DataFrame(columns=['movieId','genres'])

In [30]:
#dummy variables for checking time 
a1 = [10,100,1000,3000,5000,10000,15000,20000,25000]
b1 = 0

In [32]:
for x in movies_genres_df['movieId'].tolist():
    b1 += 1
    if b1 in a1: print(b1, str(datetime.datetime.now()))
    movies_genres=movies_genres.append(get_movie_genres(x))

(10, '2018-10-25 20:28:23.328794')
(100, '2018-10-25 20:28:23.572024')
(1000, '2018-10-25 20:28:25.911287')
(3000, '2018-10-25 20:28:31.412471')
(5000, '2018-10-25 20:28:37.363342')
(10000, '2018-10-25 20:28:55.098351')
(15000, '2018-10-25 20:29:18.001631')
(20000, '2018-10-25 20:29:46.728265')
(25000, '2018-10-25 20:30:19.224965')


In [33]:
movies_genres.head()

,movieId,genres
0,1,Adventure
1,1,Animation
2,1,Children
3,1,Comedy
4,1,Fantasy


In [34]:
movies_genres.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54406 entries, 0 to 2
Data columns (total 2 columns):
movieId    54406 non-null object
genres     54406 non-null object
dtypes: object(2)
memory usage: 1.2+ MB


In [35]:
#save movies genres data
movies_genres.to_csv('graphdb/ml_movies_genres.csv', sep='|', header=True, index=False)

# ml_users_genres

In [43]:
#join to movies data to get genre information
user_genres_df = ratings_data.merge(movies_data, left_on='movieId', right_on='movieId', how='left')

In [44]:
user_genres_df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,1112486027,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,1112484676,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,1112484819,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,1112484727,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,1112484580,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [45]:
#drop columns that are not used
user_genres_df.drop(['movieId','rating','timestamp','title'], axis = 1, inplace=True)

In [46]:
user_genres_df.head()

,userId,genres
0,1,Adventure|Children|Fantasy
1,1,Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,Mystery|Sci-Fi|Thriller
3,1,Mystery|Thriller
4,1,Crime|Mystery|Thriller


In [47]:
user_genres_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000263 entries, 0 to 20000262
Data columns (total 2 columns):
userId    int64
genres    object
dtypes: int64(1), object(1)
memory usage: 457.8+ MB


In [48]:
#define a funtion to get the most genre, it is based on counts of genre per user
def get_popular_genre(userId):
    user = user_genres_df[user_genres_df['userId']==userId]
    genres = user['genres'].tolist()
    movie_list = [b for a in [i.split('|') for i in genres] for b in a]
    counter = Counter(movie_list)
    return counter.most_common(1)[0][0]

In [56]:
get_popular_genre(10)

'Drama'

In [50]:
#create empty df
users_genres = pd.DataFrame(columns=['userId','genre'])

In [51]:
#dummy variables for checking time 
a2 = [10,100,1000,5000,10000,25000,50000,75000,100000, 125000]
b2 = 0

In [57]:
for x in user_df['userId'].tolist():
    b2 += 1
    if b2 in a2: print(b2, str(datetime.datetime.now()))
    users_genres=users_genres.append(pd.DataFrame([[x,get_popular_genre(x)]], columns=['userId','genre']))

(10, '2018-10-25 20:48:07.762940')
(100, '2018-10-25 20:48:10.118685')
(1000, '2018-10-25 20:48:33.708111')
(5000, '2018-10-25 20:50:18.077357')
(10000, '2018-10-25 20:52:30.325936')
(25000, '2018-10-25 20:59:18.987547')
(50000, '2018-10-25 21:12:11.838419')
(75000, '2018-10-25 21:27:17.951682')
(100000, '2018-10-25 21:44:01.377284')
(125000, '2018-10-25 22:00:36.274897')


In [58]:
users_genres.head()

,userId,genre
0,1,Adventure
0,2,Sci-Fi
0,3,Sci-Fi
0,4,Action
0,5,Drama


In [59]:
#save users genres data
users_genres.to_csv('graphdb/ml_users_genres.csv', sep='|', header=True, index=False)

# ml_movies_similarity

This is data very important. We will create 3 df (mov_tag_df, mov_genres_df, mov_rating_df) and calculate 3 cosine similarity then we will mix them. While we will mix the data sets, we will use (mov_tag_df*0.5+mov_genres_df*0.25+mov_rating_df*0.25). In this case tags are the most important data to calculate similarity so it effects to similarity calculation nore than others.

mov_tag_df

mov_tag_df is created with using "genome_scores.csv". We will pivot data and compare movies with tags.

In [5]:
scores_pivot = genome_scores_data.pivot_table(index = ["movieId"],columns = ["tagId"],values = "relevance").reset_index()

In [6]:
scores_pivot.head()

tagId,movieId,1,2,3,4,5,6,7,8,9,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,1,0.02500,0.02500,0.05775,0.09675,0.14675,0.21700,0.06700,0.26275,0.26200,...,0.03950,0.01800,0.04575,0.03275,0.12500,0.04150,0.01925,0.03625,0.07775,0.02300
1,2,0.03975,0.04375,0.03775,0.04800,0.11025,0.07250,0.04775,0.10975,0.09925,...,0.04175,0.01925,0.01725,0.02425,0.12550,0.02250,0.01550,0.01475,0.09025,0.01875
2,3,0.04350,0.05475,0.02800,0.07700,0.05400,0.06850,0.05600,0.18500,0.04925,...,0.04150,0.02675,0.02775,0.03425,0.15550,0.03675,0.01700,0.01950,0.09700,0.01850
3,4,0.03725,0.03950,0.03675,0.03100,0.06825,0.04050,0.02325,0.08700,0.05125,...,0.05750,0.03375,0.02275,0.03975,0.18525,0.05925,0.01500,0.01525,0.06450,0.01300
4,5,0.04200,0.05275,0.05925,0.03675,0.07525,0.12525,0.02850,0.08500,0.02950,...,0.04250,0.02825,0.02150,0.02600,0.14275,0.02075,0.01650,0.01675,0.10750,0.01825


In [7]:
scores_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10381 entries, 0 to 10380
Columns: 1129 entries, movieId to 1128
dtypes: float64(1128), int64(1)
memory usage: 89.4 MB


In [8]:
#join with movies data to get all movieIds 
mov_tag_df = movies_data.merge(scores_pivot, left_on='movieId', right_on='movieId', how='left')

In [9]:
#fill null values and drop columns that are not used
mov_tag_df = mov_tag_df.fillna(0)
mov_tag_df = mov_tag_df.drop(['title','genres'], axis = 1)

In [10]:
mov_tag_df.head()

,movieId,1,2,3,4,5,6,7,8,9,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128
0,1,0.02500,0.02500,0.05775,0.09675,0.14675,0.21700,0.06700,0.26275,0.26200,...,0.03950,0.01800,0.04575,0.03275,0.12500,0.04150,0.01925,0.03625,0.07775,0.02300
1,2,0.03975,0.04375,0.03775,0.04800,0.11025,0.07250,0.04775,0.10975,0.09925,...,0.04175,0.01925,0.01725,0.02425,0.12550,0.02250,0.01550,0.01475,0.09025,0.01875
2,3,0.04350,0.05475,0.02800,0.07700,0.05400,0.06850,0.05600,0.18500,0.04925,...,0.04150,0.02675,0.02775,0.03425,0.15550,0.03675,0.01700,0.01950,0.09700,0.01850
3,4,0.03725,0.03950,0.03675,0.03100,0.06825,0.04050,0.02325,0.08700,0.05125,...,0.05750,0.03375,0.02275,0.03975,0.18525,0.05925,0.01500,0.01525,0.06450,0.01300
4,5,0.04200,0.05275,0.05925,0.03675,0.07525,0.12525,0.02850,0.08500,0.02950,...,0.04250,0.02825,0.02150,0.02600,0.14275,0.02075,0.01650,0.01675,0.10750,0.01825


In [11]:
mov_tag_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27278 entries, 0 to 27277
Columns: 1129 entries, movieId to 1128
dtypes: float64(1128), int64(1)
memory usage: 235.2 MB


mov_genres_df

mov_genres_df is created with using "movies.csv". We split genres for movies. Then we create columns for each genres

In [12]:
mov_genres_df = movies_data.drop('title', axis = 1)

In [13]:
mov_genres_df.head()

,movieId,genres
0,1,Adventure|Animation|Children|Comedy|Fantasy
1,2,Adventure|Children|Fantasy
2,3,Comedy|Romance
3,4,Comedy|Drama|Romance
4,5,Comedy


In [14]:
#define function to set genders column if exists or not
def set_genres(genres,col):
    if genres in col.split('|'): return 1
    else: return 0

In [15]:
mov_genres_df["Action"] = mov_genres_df.apply(lambda x: set_genres("Action",x['genres']), axis=1)
mov_genres_df["Adventure"] = mov_genres_df.apply(lambda x: set_genres("Adventure",x['genres']), axis=1)
mov_genres_df["Animation"] = mov_genres_df.apply(lambda x: set_genres("Animation",x['genres']), axis=1)
mov_genres_df["Children"] = mov_genres_df.apply(lambda x: set_genres("Children",x['genres']), axis=1)
mov_genres_df["Comedy"] = mov_genres_df.apply(lambda x: set_genres("Comedy",x['genres']), axis=1)
mov_genres_df["Crime"] = mov_genres_df.apply(lambda x: set_genres("Crime",x['genres']), axis=1)
mov_genres_df["Documentary"] = mov_genres_df.apply(lambda x: set_genres("Documentary",x['genres']), axis=1)
mov_genres_df["Drama"] = mov_genres_df.apply(lambda x: set_genres("Drama",x['genres']), axis=1)
mov_genres_df["Fantasy"] = mov_genres_df.apply(lambda x: set_genres("Fantasy",x['genres']), axis=1)
mov_genres_df["Film-Noir"] = mov_genres_df.apply(lambda x: set_genres("Film-Noir",x['genres']), axis=1)
mov_genres_df["Horror"] = mov_genres_df.apply(lambda x: set_genres("Horror",x['genres']), axis=1)
mov_genres_df["Musical"] = mov_genres_df.apply(lambda x: set_genres("Musical",x['genres']), axis=1)
mov_genres_df["Mystery"] = mov_genres_df.apply(lambda x: set_genres("Mystery",x['genres']), axis=1)
mov_genres_df["Romance"] = mov_genres_df.apply(lambda x: set_genres("Romance",x['genres']), axis=1)
mov_genres_df["Sci-Fi"] = mov_genres_df.apply(lambda x: set_genres("Sci-Fi",x['genres']), axis=1)
mov_genres_df["Thriller"] = mov_genres_df.apply(lambda x: set_genres("Thriller",x['genres']), axis=1)
mov_genres_df["War"] = mov_genres_df.apply(lambda x: set_genres("War",x['genres']), axis=1)
mov_genres_df["Western"] = mov_genres_df.apply(lambda x: set_genres("Western",x['genres']), axis=1)
mov_genres_df["(no genres listed)"] = mov_genres_df.apply(lambda x: set_genres("(no genres listed)",x['genres']), axis=1)

In [16]:
#not need genres anymore
mov_genres_df.drop('genres', axis = 1, inplace=True)

In [17]:
mov_genres_df.head()

,movieId,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,(no genres listed)
0,1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
mov_genres_df.sum()

movieId               1632737799
Action                      3520
Adventure                   2329
Animation                   1027
Children                    1139
Comedy                      8374
Crime                       2939
Documentary                 2471
Drama                      13344
Fantasy                     1412
Film-Noir                    330
Horror                      2611
Musical                     1036
Mystery                     1514
Romance                     4127
Sci-Fi                      1743
Thriller                    4178
War                         1194
Western                      676
(no genres listed)           246
dtype: int64

mov_rating_df

mov_rating_df includes year, rating and rating count information. We make same calculations. First we extract year information from title field. Then we group years between 0-5. We calculate mean of rating for each movie. We also calculate counts of ratings for ech movie. Then we group rating counts between 0-5. We group years and rating counts because reduce the scale. It helps to calculate better similarity 

In [19]:
movies = movies_data.drop('genres', axis = 1)

In [20]:
#define function to extract year
def set_year(title):
    year = title.strip()[-5:-1]
    if unicode(year, 'utf-8').isnumeric() == True: return int(year)
    else: return 1800

In [21]:
#add year field
movies['year'] = movies.apply(lambda x: set_year(x['title']), axis=1)

In [22]:
movies.head()

,movieId,title,year
0,1,Toy Story (1995),1995
1,2,Jumanji (1995),1995
2,3,Grumpier Old Men (1995),1995
3,4,Waiting to Exhale (1995),1995
4,5,Father of the Bride Part II (1995),1995


In [23]:
#define function to group years
def set_year_group(year):
    if (year < 1900): return 0
    elif (1900 <= year <= 1975): return 1
    elif (1976 <= year <= 1995): return 2
    elif (1996 <= year <= 2003): return 3
    elif (2004 <= year <= 2009): return 4
    elif (2010 <= year): return 5
    else: return 0

In [24]:
movies['year_group'] = movies.apply(lambda x: set_year_group(x['year']), axis=1)

In [25]:
movies.head()

,movieId,title,year,year_group
0,1,Toy Story (1995),1995,2
1,2,Jumanji (1995),1995,2
2,3,Grumpier Old Men (1995),1995,2
3,4,Waiting to Exhale (1995),1995,2
4,5,Father of the Bride Part II (1995),1995,2


In [26]:
#no need title and year fields
movies.drop(['title','year'], axis = 1, inplace=True)

In [27]:
#calculate mean and counts of ratings for each movies
agg_movies_rat = ratings_data.groupby(['movieId']).agg({'rating': [np.size, np.mean]}).reset_index()

In [28]:
agg_movies_rat.columns = ['movieId','rating_counts', 'rating_mean']

In [29]:
agg_movies_rat.head()

,movieId,rating_counts,rating_mean
0,1,49695.0,3.921240
1,2,22243.0,3.211977
2,3,12735.0,3.151040
3,4,2756.0,2.861393
4,5,12161.0,3.064592


In [30]:
#define function to group rating counts
def set_rating_group(rating_counts):
    if (rating_counts <= 1): return 0
    elif (2 <= rating_counts <= 10): return 1
    elif (11 <= rating_counts <= 100): return 2
    elif (101 <= rating_counts <= 1000): return 3
    elif (1001 <= rating_counts <= 5000): return 4
    elif (5001 <= rating_counts): return 5
    else: return 0

In [31]:
agg_movies_rat['rating_group'] = agg_movies_rat.apply(lambda x: set_rating_group(x['rating_counts']), axis=1)

In [32]:
agg_movies_rat.head()

,movieId,rating_counts,rating_mean,rating_group
0,1,49695.0,3.921240,5
1,2,22243.0,3.211977,5
2,3,12735.0,3.151040,5
3,4,2756.0,2.861393,4
4,5,12161.0,3.064592,5


In [33]:
#no need rating_counts field
agg_movies_rat.drop('rating_counts', axis = 1, inplace=True)

In [34]:
mov_rating_df = movies.merge(agg_movies_rat, left_on='movieId', right_on='movieId', how='left')

In [35]:
mov_rating_df.head()

,movieId,year_group,rating_mean,rating_group
0,1,2,3.921240,5.0
1,2,2,3.211977,5.0
2,3,2,3.151040,5.0
3,4,2,2.861393,4.0
4,5,2,3.064592,5.0


In [36]:
mov_rating_df = mov_rating_df.fillna(0)

In [37]:
mov_rating_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27278 entries, 0 to 27277
Data columns (total 4 columns):
movieId         27278 non-null int64
year_group      27278 non-null int64
rating_mean     27278 non-null float64
rating_group    27278 non-null float64
dtypes: float64(2), int64(2)
memory usage: 1.0 MB


calculate similarity with using cosine similarity function

In [38]:
#before calculate cosine similarity, set movieId field as index
mov_tag_df = mov_tag_df.set_index('movieId')
mov_genres_df = mov_genres_df.set_index('movieId')
mov_rating_df = mov_rating_df.set_index('movieId')

In [39]:
#cosine similarity for mov_tag_df
cos_tag = cosine_similarity(mov_tag_df.values)*0.5

In [40]:
#cosine similarity for mov_tag_df
cos_genres = cosine_similarity(mov_genres_df.values)*0.25

In [41]:
#cosine similarity for mov_tag_df
cos_rating = cosine_similarity(mov_rating_df.values)*0.25

In [42]:
#mix
cos = cos_tag+cos_genres+cos_rating

In [43]:
#create df
cols = mov_tag_df.index.values
inx = mov_tag_df.index
movies_sim = pd.DataFrame(cos, columns=cols, index=inx)

In [44]:
movies_sim.head()

,1,2,3,4,5,6,7,8,9,10,...,131241,131243,131248,131250,131252,131254,131256,131258,131260,131262
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.829784,0.680520,0.640024,0.695793,0.561475,0.653804,0.766840,0.519063,0.631729,...,0.241819,0.320876,0.407131,0.274566,0.241819,0.268935,0.274566,0.244749,0.157131,0.279640
2,0.829784,1.000000,0.593250,0.568572,0.589406,0.510859,0.568324,0.840497,0.569749,0.661595,...,0.150295,0.150295,0.340589,0.150295,0.150295,0.146939,0.150295,0.272931,0.146939,0.308934
3,0.680520,0.593250,1.000000,0.833609,0.857064,0.570108,0.888614,0.610713,0.558890,0.587168,...,0.399088,0.274088,0.225000,0.325865,0.274088,0.322720,0.325865,0.128141,0.145943,0.141449
4,0.640024,0.568572,0.833609,1.000000,0.767326,0.554754,0.863996,0.623618,0.540909,0.549538,...,0.368420,0.266358,0.226416,0.308633,0.266358,0.306204,0.308633,0.144490,0.161866,0.157709
5,0.695793,0.589406,0.857064,0.767326,1.000000,0.511446,0.817490,0.584793,0.538056,0.548123,...,0.324114,0.324114,0.256298,0.397337,0.324114,0.394494,0.397337,0.127474,0.144494,0.140256


In [45]:
#define function to extract the most 5 similar movies for each movies
def get_similar(movieId):
    df = movies_sim.loc[movies_sim.index == movieId].reset_index(). \
            melt(id_vars='movieId', var_name='sim_moveId', value_name='relevance'). \
            sort_values('relevance', axis=0, ascending=False)[1:6]
    return df 

In [46]:
#create empty df
ml_movies_similarity = pd.DataFrame(columns=['movieId','sim_moveId','relevance'])

In [47]:
#dummy variables for checking time 
a3 = [10,100,1000,5000,10000,20000,30000]
b3 = 0

In [49]:
for x in movies_sim.index.tolist():
    b3 += 1
    if b3 in a3: print(b3, str(datetime.datetime.now()))
    ml_movies_similarity=ml_movies_similarity.append(get_similar(x))

(10, '2018-10-25 23:31:06.055935')
(100, '2018-10-25 23:31:08.017556')
(1000, '2018-10-25 23:31:29.636927')
(5000, '2018-10-25 23:33:30.255058')
(10000, '2018-10-25 23:36:34.289143')
(20000, '2018-10-25 23:44:01.762753')


In [50]:
ml_movies_similarity.head()

,movieId,sim_moveId,relevance
4790,1,4886,0.977045
3027,1,3114,0.975334
2209,1,2294,0.946482
2270,1,2355,0.945159
15401,1,78499,0.944199


In [51]:
ml_movies_similarity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136390 entries, 4790 to 15489
Data columns (total 3 columns):
movieId       136390 non-null object
sim_moveId    136390 non-null object
relevance     136390 non-null float64
dtypes: float64(1), object(2)
memory usage: 4.2+ MB


In [52]:
#save users genres data
ml_movies_similarity.to_csv('graphdb/ml_movies_similarity.csv', sep='|', header=True, index=False)

# recommendation

Data is ready to import graph db. We already calculate similarity of movies so we can create a function to get 5 similar movies

In [59]:
def movie_recommender(movieId):
    df = movies_sim.loc[movies_sim.index == movieId].reset_index(). \
            melt(id_vars='movieId', var_name='sim_moveId', value_name='relevance'). \
            sort_values('relevance', axis=0, ascending=False)[1:6]
    df['sim_moveId'] = df['sim_moveId'].astype(int)
    sim_df = movies_data.merge(df, left_on='movieId', right_on='sim_moveId', how='inner'). \
                sort_values('relevance', axis=0, ascending=False). \
                loc[: , ['movieId_y','title','genres']]. \
                rename(columns={ 'movieId_y': "movieId" })
    return sim_df 

In [66]:
#check movieId 1
movies_data[movies_data['movieId'] == 3793]

,movieId,title,genres
3702,3793,X-Men (2000),Action|Adventure|Sci-Fi


In [61]:
#get recommendation for Toy Story
movie_recommender(1)

,movieId,title,genres
3,1,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy
2,1,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy
0,1,Antz (1998),Adventure|Animation|Children|Comedy|Fantasy
1,1,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy
4,1,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX


In [63]:
#get recommendation for Inception
movie_recommender(79132)

,movieId,title,genres
4,79132,Source Code (2011),Action|Drama|Mystery|Sci-Fi|Thriller
1,79132,Minority Report (2002),Action|Crime|Mystery|Sci-Fi|Thriller
2,79132,"Prestige, The (2006)",Drama|Mystery|Sci-Fi|Thriller
0,79132,Strange Days (1995),Action|Crime|Drama|Mystery|Sci-Fi|Thriller
3,79132,Moon (2009),Drama|Mystery|Sci-Fi|Thriller


In [67]:
#get recommendation for X-Men
movie_recommender(3793)

,movieId,title,genres
1,3793,Spider-Man 2 (2004),Action|Adventure|Sci-Fi|IMAX
2,3793,Iron Man (2008),Action|Adventure|Sci-Fi
0,3793,X2: X-Men United (2003),Action|Adventure|Sci-Fi|Thriller
3,3793,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX
4,3793,X-Men: Days of Future Past (2014),Action|Adventure|Sci-Fi


In [68]:
#get recommendation for Lock, Stock & Two Smoking Barrels
movie_recommender(2542)

,movieId,title,genres
2,2542,Snatch (2000),Comedy|Crime|Thriller
0,2542,Get Shorty (1995),Comedy|Crime|Thriller
1,2542,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
4,2542,In Bruges (2008),Comedy|Crime|Drama|Thriller
3,2542,Two Hands (1999),Comedy|Crime|Thriller


In [69]:
#get recommendation for Casino Royale
movie_recommender(49272)

,movieId,title,genres
4,49272,Skyfall (2012),Action|Adventure|Thriller|IMAX
3,49272,Quantum of Solace (2008),Action|Adventure|Thriller
2,49272,"World Is Not Enough, The (1999)",Action|Adventure|Thriller
0,49272,Tomorrow Never Dies (1997),Action|Adventure|Thriller
1,49272,From Russia with Love (1963),Action|Adventure|Thriller


In [71]:
#get recommendation for Hangover Part II
movie_recommender(86911)

,movieId,title,genres
3,86911,"Hangover Part III, The (2013)",Comedy
4,86911,Dumb and Dumber To (2014),Comedy
2,86911,Hall Pass (2011),Comedy
1,86911,Meet the Fockers (2004),Comedy
0,86911,Next Friday (2000),Comedy


In [72]:
#get recommendation for Eternal Sunshine of the Spotless Mind
movie_recommender(7361)

,movieId,title,genres
4,7361,Her (2013),Drama|Romance|Sci-Fi
1,7361,Mr. Nobody (2009),Drama|Fantasy|Romance|Sci-Fi
2,7361,Never Let Me Go (2010),Drama|Romance|Sci-Fi
3,7361,Another Earth (2011),Drama|Romance|Sci-Fi
0,7361,Open Your Eyes (Abre los ojos) (1997),Drama|Romance|Sci-Fi|Thriller


In [73]:
#get recommendation for Scream 4
movie_recommender(86295)

,movieId,title,genres
3,86295,Scream 3 (2000),Comedy|Horror|Mystery|Thriller
1,86295,Scream 2 (1997),Comedy|Horror|Mystery|Thriller
0,86295,Scream (1996),Comedy|Horror|Mystery|Thriller
4,86295,Final Destination 3 (2006),Horror|Mystery|Thriller
2,86295,Bride of Chucky (Child's Play 4) (1998),Comedy|Horror|Thriller


I think recommendation system works well! :)